# Knowledge Distillation on MNIST
Knowledge distillation is the process of transferring the higher performance of a more expensive model to a smaller one.  In this notebook, we will explore performing this process on MNIST.  To begin with, I have provided access to pre-trained model that is large, but performant.  The exact architecture is not relevant (although you can inspect this easily if you wish).  It is straightforward to load in pytorch with

In [1]:
import torch
device = 'cpu'

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(28**2,800)
        self.l2 = torch.nn.Linear(800,800)
        self.l3 = torch.nn.Linear(800,10)
        self.dropout2 = torch.nn.Dropout(0.5)
        self.dropout3 = torch.nn.Dropout(0.5)

    def forward(self, x):
        x = self.l1(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        x = self.l2(x)
        x = torch.relu(x)
        x = self.dropout3(x)
        x = self.l3(x)
        return x
    
big_model = torch.load('pretrained_model.pt').to(device)

First, let's establish the baseline performance of the big model on the MNIST test set.  Of course we'll need acces to the MNIST test set to do this.  At the same time, let's also get our transfer set, which in this case will be a $n=10$k subset of the full MNIST training set (using a subset is helpful for speeding up training of distilled models, and also helps showcase some of the improved performance due to model distillation).   

In [2]:
from torchvision import transforms, datasets
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    ])

dataset_train = datasets.MNIST('./data', train=True, download=True, transform=transform)

dataset_test = datasets.MNIST('../data', train=False, download=True, transform=transform)

# This is a useful function that I didn't know about before
first_10k = list(range(0, 10000))
dataset_transfer = torch.utils.data.Subset(dataset_train, first_10k)

batch_size = 32
num_workers = 4
transfer_loader = torch.utils.data.DataLoader(dataset_transfer,batch_size=batch_size,num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(dataset_test,batch_size=batch_size,num_workers=num_workers)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



Here's a function that runs the big model in test mode and provides the number of correct examples

In [3]:
def test(model,test_loader):
    correct = 0
    counter = 0
    model.eval()
    with torch.no_grad():
        for data,target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.reshape(data.shape[0],-1)
            logits = model(data)
            pred = logits.argmax(dim=1,keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            counter += batch_size
    return correct

test(big_model,test_loader)

9833

We find that the big model gets 167 examples wrong (not quite as good as the Hinton paper, but who cares). 

Now we would like to perform knowledge distillation by training a smaller model to approximate the larger model's performance on the transfer set.  First, let's build a smaller model.  You may use whatever architecture you choose, but I found that using two hidden layers, each with 200 units along with ReLU activations (and no regularization at all) worked fine.

In [25]:
class SmallNet(torch.nn.Module):
    def __init__(self):
        super(SmallNet, self).__init__()
        self.l1 = torch.nn.Linear(28**2, 200)
        self.l2 = torch.nn.Linear(200,200)
        self.l3 = torch.nn.Linear(200,10)

    def forward(self, x):
        x = self.l1(x)
        x = torch.relu(x)
        x = self.l2(x)
        x = torch.relu(x)
        x = self.l3(x)
        return x
    
small_model = SmallNet()
small_model.to(device)

SmallNet(
  (l1): Linear(in_features=784, out_features=200, bias=True)
  (l2): Linear(in_features=200, out_features=200, bias=True)
  (l3): Linear(in_features=200, out_features=10, bias=True)
)

**To establish a baseline performance level, train the small model on the transfer set**  

In [26]:
# I'm giving you this training function: you'll need to modify it below to do knowledge distillation
def train(model,train_loader,n_epochs):
    optimizer = torch.optim.Adam(model.parameters(),1e-3)
    loss_fn = torch.nn.CrossEntropyLoss()
    model.train()
    for epoch in range(n_epochs):
        avg_l = 0.0
        counter = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            data = data.reshape(data.shape[0],-1)
            optimizer.zero_grad()
            logits = model(data)
            L = loss_fn(logits,target)
            L.backward()
            optimizer.step()
            with torch.no_grad():
                avg_l += L
                counter += 1
        print(epoch,avg_l/counter)

train(small_model,transfer_loader,50)

0 tensor(0.4511)
1 tensor(0.1890)
2 tensor(0.1202)
3 tensor(0.0856)
4 tensor(0.0597)
5 tensor(0.0455)
6 tensor(0.0404)
7 tensor(0.0323)
8 tensor(0.0362)
9 tensor(0.0311)
10 tensor(0.0194)
11 tensor(0.0186)
12 tensor(0.0338)
13 tensor(0.0217)
14 tensor(0.0106)
15 tensor(0.0170)
16 tensor(0.0100)
17 tensor(0.0256)
18 tensor(0.0240)
19 tensor(0.0265)
20 tensor(0.0162)
21 tensor(0.0060)
22 tensor(0.0028)
23 tensor(0.0403)
24 tensor(0.0100)
25 tensor(0.0274)
26 tensor(0.0145)
27 tensor(0.0059)
28 tensor(0.0129)
29 tensor(0.0119)
30 tensor(0.0116)
31 tensor(0.0150)
32 tensor(0.0145)
33 tensor(0.0231)
34 tensor(0.0050)
35 tensor(0.0019)
36 tensor(0.0099)
37 tensor(0.0128)
38 tensor(0.0119)
39 tensor(0.0180)
40 tensor(0.0191)
41 tensor(0.0219)
42 tensor(0.0012)
43 tensor(8.8503e-05)
44 tensor(3.2778e-05)
45 tensor(2.5941e-05)
46 tensor(2.1452e-05)
47 tensor(1.8104e-05)
48 tensor(1.5472e-05)
49 tensor(1.3298e-05)


**Evaluate the small model on the test set, and comment on its accuracy relative to the big model.**  As you might expect, the performance is relatively worse.  

In [27]:
test(small_model, test_loader)

9621

**The primary task of this notebook is now as follows: create a new training function similar to "train" above, but instead called "distill".**  "distill" should perform knowledge distillation as outlined in this week's paper.  It should accept a few additional arguments compared to train, namely the big model, the temperature hyperparameter, and a hyperparameter $\alpha$ that weights the relative magnitude of the soft target loss and the hard target loss.

In [56]:
distilled_model = SmallNet()
distilled_model.to(device)

# The body of this method is currently copied verbatim from the train method above: 
# you will need to modify it to utilize the big_model, temperature, and alpha values 
# to perform knowledge distillation
def distill(small_model,big_model,T,alpha,transfer_loader,n_epochs):
    optimizer = torch.optim.Adam(small_model.parameters(),1e-3)
    loss_fn = torch.nn.CrossEntropyLoss()
    soft_fn = torch.nn.Softmax()
    small_model.train()
    for epoch in range(n_epochs):
        avg_l = 0.0
        counter = 0
        for batch_idx, (data, target) in enumerate(transfer_loader):
            data, target = data.to(device), target.to(device)
            data = data.reshape(data.shape[0],-1)
            optimizer.zero_grad()
            logits = small_model(data)
            with torch.no_grad():
                big_logits = big_model(data)
            soft_target = soft_fn(big_logits/T)
            L1 = loss_fn(logits,target)
            L2 = loss_fn(logits/T, soft_target)
            L = (alpha*L1) + ((1-alpha)*L2)
            L.backward()
            optimizer.step()
            with torch.no_grad():
                avg_l += L
                counter += 1
        print(epoch,avg_l/counter)

alpha = 0.2
T = 3
        
distill(distilled_model,big_model,T,alpha,transfer_loader,50)

/tmp/ipykernel_1420187/1532496489.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  soft_target = soft_fn(big_logits/T)


0 tensor(0.5948)
1 tensor(0.3039)
2 tensor(0.2282)
3 tensor(0.1852)
4 tensor(0.1602)
5 tensor(0.1435)
6 tensor(0.1308)
7 tensor(0.1239)
8 tensor(0.1218)
9 tensor(0.1228)
10 tensor(0.1209)
11 tensor(0.1218)
12 tensor(0.1206)
13 tensor(0.1151)
14 tensor(0.1121)
15 tensor(0.1158)
16 tensor(0.1134)
17 tensor(0.1173)
18 tensor(0.1180)
19 tensor(0.1152)
20 tensor(0.1162)
21 tensor(0.1112)
22 tensor(0.1128)
23 tensor(0.1100)
24 tensor(0.1062)
25 tensor(0.1056)
26 tensor(0.1067)
27 tensor(0.1061)
28 tensor(0.1051)
29 tensor(0.1059)
30 tensor(0.1073)
31 tensor(0.1075)
32 tensor(0.1069)
33 tensor(0.1131)
34 tensor(0.1065)
35 tensor(0.1037)
36 tensor(0.1025)
37 tensor(0.1024)
38 tensor(0.1027)
39 tensor(0.1023)
40 tensor(0.1032)
41 tensor(0.1063)
42 tensor(0.1059)
43 tensor(0.1156)
44 tensor(0.1064)
45 tensor(0.1019)
46 tensor(0.1005)
47 tensor(0.0997)
48 tensor(0.0992)
49 tensor(0.0994)


**Finally, test your distilled model (on the test set) and describe how it performs relative to both big and small models.**

*After playing with the temperature and weight parameters, I'm able to get the distilled model performance up to at most ~9725 correct, compared to 9620 and 9830 for the small and big models respectively. This is more or less as expected (though I would have like to see it closer to the big model performance). The distilled model should outperform the small model alone, since it's given the advantage of the learning from the big model. However, it only sees a tiny portion of the data, has fewer parameters, and doesn't run as long as the big model, so it's not surprising that it doesn't quite match in performance.*

In [57]:
test(distilled_model,test_loader)

9710